# Chapter 13. Text Processing

The chapter focuses on classical string algorithms (Boyer–Moore, Knuth–Morris–Pratt, etc.), and also introduces the concept of dynamic programming.

## Importand Algorithms and Data Structures

In [2]:
# Substring Search, brute force search
def brute(T, P):
    n = len(T) - len(P) + 1
    for i in range(n):
        j = 0
        while j < len(P):
            if P[j] != T[i+j]:
                break
            j += 1
        if j == len(P):
            return i
    return -1

# text book simplified version of the Boyer–Moore string-search algorithm 
# which results in O(nm) worst case, the same as brute force, but on average it's better
# m - length of the substring, n - length of the whole string
def boyer_moore(s, sub_s):
    n, m = len(s), len(sub_s)
    last = {}
    for i in range(m):
        last[ sub_s[i] ] = i
    i = m - 1
    j = m - 1
    while i < n:
        if s[i] == sub_s[j]:
            if j == 0:
                return i
            else:
                i -= 1
                j -= 1
        else:
            k = last.get(s[i], -1)
            i += m - min(j, k + 1) # very smart trick here which saves infinitely many lines of code
            j = m - 1
    return -1

# Knuth–Morris–Pratt string searching algorithm, O(n+m) time complexity
def fail_function(A):
    n = len(A)
    fail = [0] * n
    j = 1
    k = 0
    while j < n:
        if A[j] == A[k]:
            fail[j] = k + 1
            j += 1
            k += 1
        elif k > 0:
            k = fail[k-1] # OK, I've tried many times but still didn't get it, we could use k=0 and it still works...
        else:
            j += 1
    return fail

def kpm(T, P):
    n, m = len(T), len(P)
    if m == 0:
        return 0
    fail = fail_function(P)
    j = 0
    k = 0
    while j < n:
        if T[j] == P[k]:
            if k == m-1:
                return j - m + 1
            j += 1
            k += 1
        elif k > 0:
            k = fail[k-1]
        else:
            j += 1
    return -1

# Longest Common Subsequence problem

# brute force version, exponential time
def lcs_brute(S1, S2, i=0, j=0):
    if i == len(S1) or j == len(S2):
        return 0
    elif S1[i] == S2[j]:
        return 1 + lcs(S1, S2, i+1, j+1)
    else:
        return max(lcs(S1, S2, i, j+1), lcs(S1, S2, i+1))

# dynamic programmin silution, O(nm) time-complexity
def lcs(S1, S2):
    n, m = len(S1), len(S2)
    table = [[0]*(m+1) for i in range(n+1)]
    for i in range(n):
        for j in range(m):
            if S1[i] == S2[j]:
                table[i+1][j+1] = 1 + table[i][j]
            else:
                table[i+1][j+1] = max(table[i+1][j], table[i][j+1])
    return table[-1][-1]


## Reinforcement 

### R-13.1

List the prefixes of the string ${P=}$ "${aaabbaaa}$" that are also suffixes of ${P}$.

In [3]:
# Suf = a, aa, aaa, baaa, bbaaa, abbaaa, aabbaaa, aaabbaaa
# pref = a, aa, aaa, aaa, aaab, aaabb, aaabba, aaabbaa, aaabbaaa
# common: a, aa, aaa, aaabbaaa
# the difference that it's just a prefix/suffix, which means it could include the whole word
# proper suffix/prefix is maximum len(word) - 1 


### R-13.2

What is the longest (proper) prefix of the string "${cgtacgttcgtacg}$" that is also a suffix of this string?

In [4]:
# cgtacgttcgtacg suf = g, cg, acg, tacg, gtacg, cgtacg(!), tcgtacg, ttcgtacg, gttcgtacg
# cgttcgtacg, acgttcgtacg, tacgttcgtacg, gtacgttcgtacg

# pref = c, cg, cgt, cgta, cgtac, cgtacg(!), cgtacgt, cgtacgtt, cgtacgttc, cgtacgttcg, cgtacgttcgt,
# cgtacgttcgta, cgtacgttcgtac

# largest common - cgtacg


### R-13.7

Compute a table representing the Knuth-Morris-Pratt failure function for the pattern string "${cgtacgttcgtac}$".

In [5]:
# c g t a c g t t c g t a c
# 0 0 0 0 1 2 3 0 1 2 3 4 5


## Creativity 

### C-13.16

Adapt the brute-force pattern-matching algorithm in order to implement a function, rfind_brute(T,P), that returns the index at which the rightmost occurrence of pattern ${P}$ within text ${T}$, if any.

In [6]:
def rfind_brute(T, P):
    index = -1
    n = len(T) - len(P) + 1
    for i in range(n):
        j = 0
        while j < len(P):
            if P[j] != T[i+j]:
                break
            j += 1
        if j == len(P):
            index = i
    return index


### C-13.18

Redo Exercise C-13.16, adapting the Knuth-Morris-Pratt pattern-matching algorithm appropriately to implement a function rfind_kmp(T,P).

In [8]:
def rfind_kpm(T, P):
    index = -1
    n, m = len(T), len(P)
    if m == 0:
        return 0
    fail = fail_function(P)
    j = 0
    k = 0
    while j < n:
        if T[j] == P[k]:
            if k == m-1:
                index = j - m + 1
                j+= 1
                k = 0
            else:
                j += 1
                k += 1
        elif k > 0:
            k = fail[k-1]
        else:
            j += 1
    return index


### C-13.19

The count method of Python’s str class reports the maximum number of nonoverlapping occurrences of a pattern within a string. For example, the call 'abababa'.count('aba') returns 2 (not 3). Adapt the brute-force pattern-matching algorithm to implement a function, count_brute(T,P), with similar outcome.

In [10]:
def count_brute(T, P):
    counter = 0 
    n = len(T) - len(P) + 1
    i = 0
    while i < n:
        j = 0
        while j < len(P):
            if P[j] != T[i+j]:
                break
            j += 1
        if j == len(P):
            counter += 1
            i += len(P)
        else:
            i+= 1
    return counter


### C-13.21

Redo Exercise C-13.19, adapting the Knuth-Morris-Pratt pattern-matching lgorithm appropriately to implement a function count_kmp(T,P).

In [11]:
def count_kpm(T, P):
    counter = 0
    n, m = len(T), len(P)
    if m == 0:
        return 0
    fail = fail_function(P)
    j = 0
    k = 0
    while j < n:
        if T[j] == P[k]:
            if k == m-1:
                counter += 1
                j += 1
                continue
            j += 1
            k += 1
        elif k > 0:
            k = fail[k-1]
        else:
            j += 1
    return counter


### C-13.27

Design an efficient algorithm for the matrix chain multiplication problem that outputs a fully parenthesized expression for how to multiply the matrices in the chain using the minimum number of operations.

In [17]:
# well I guess the text book aproach also works, and it has less code
# but here is mine
# time complexity: O(n^3)
# space complexity: O(n^2)

# very smart trick with loops in the middle: first compute |i - j| = 1, then |i - j| = 2, |i - j| = 3 etc.
# basically L is the number of matrixes multiplied
# input example: ex = [3, 2, 4, 2, 5] means A1(2x3) * A2(3x4) * A3(4x2), e.g. these are 4 dimensions
def matrix_chain(m):
    n = len(m)
    table = [[0]*n for i in range(n)]
    for L in range(2, n):
        for i in range(1, n-L+1):      
            j = i+L-1
            maximum = 123456789
            for k in range(i, j):
                if maximum > table[i][k] + table[k+1][j] + m[i-1]*m[k]*m[j]:
                    maximum = table[i][k] + table[k+1][j] + m[i-1]*m[k]*m[j]
            table[i][j] = maximum
    return table

ex = [3, 2, 4, 2, 5] # A1(2x3) * A2(3x4) * A3(4x2), e.g. these are 4 dimensions
print(matrix_chain(ex))

[[0, 0, 0, 0, 0], [0, 0, 24, 28, 58], [0, 0, 0, 16, 36], [0, 0, 0, 0, 40], [0, 0, 0, 0, 0]]


### C-13.29

Describe an efficient greedy algorithm for making change for a specified value using a minimum number of coins, assuming there are four denominations of coins (called quarters, dimes, nickels, and pennies), with values 25, 10, 5, and 1, respectively. Argue why your algorithm is correct.

In [13]:
# here is a greedy version
# works not for all denominations
def coins_greedy(amount, denom):
    coins = []
    j = len(denom) - 1
    while j >= 0:
        while amount >= denom[j]:
            amount -= denom[j]
            coins.append(denom[j]) 
        j -= 1
    return coins

# bonus: dynamic programming solution
# time complexity: O(A*d), where A - amount, d - number of denominations
# probably need to optimize this later 
def coins_exchange(amount, denom):
    A = [0]*(amount+1)
    max_exchange = amount + 1
    for i in range(1, len(A)):
        m = [max_exchange]*len(denom)
        for j in range(len(denom)):
            if i - denom[j] >= 0:
                m[j] = A[i-denom[j]]
            A[i] = 1 + min(m)
    return A


### C-13.30

Give an example set of denominations of coins so that a greedy changemaking algorithm will not use the minimum number of coins.

In [14]:
# example = [1, 15, 25]
# output -> 1x25 and 5x1
# most efficient -> 2x15


### C-13.35

Given a sequence ${S = (x_0,x_1, ... , x_{n−1})}$ of numbers, describe an ${O(n^2)}$ - time algorithm for finding a longest subsequence ${T = (x_{i_0} ,x_{i_1}, ... , x_{i_{k−1}})}$ of numbers, such that ${i_j < i_{j+1}}$ and ${x_{i_j} > x_{i_{j+1}}}$. That is, ${T}$ is a longest decreasing subsequence of ${S}$.

In [15]:
# okay, the 'naive' approach is to store all possible subsequences in one array
# and then choose the shortest, results in O(2^n)

# dynamic programming solution, where n = len(S)
# time-complexity: O(n^2)
# space-complexity: O(n)
def lds(S):
    table = [1]*len(S)
    for i in range(1, len(S)):
        result = 0
        for j in range(i-1, -1, -1):
            if S[i] < S[j]:
                result = 1 + table[j]
                break
        table[i] = max(1, result)
    return max(table)


### C-13.37

Define the edit distance between two strings ${X}$ and ${Y}$ of length ${n}$ and ${m}$, respectively, to be the number of edits that it takes to change ${X}$ into ${Y}$. An edit consists of a character insertion, a character deletion, or a character replacement. For example, the strings "algorithm" and "rhythm" have edit distance 6. Design an ${O(n\times m)}$-time algorithm for computing the edit distance between ${X}$ and ${Y}$.

In [16]:
# brute force approach
# time complexity: O(3^n)
def brute_ed(s1, s2, i, j):
    if i == 0: # string s1 is empty, load all remaining s2 characters there 
        return j
    if j == 0: # string s2 is empty, load all remaining s1 characters there 
        return i
    if s1[i] == s2[j]:
        return brute_ed(s1, s2, i-1, j-1) # if they are identical, just move on 
    return 1 + min(brute_ed(s1, s2, i, j-1), # insert
          brute_ed(s1, s2, i-1, j), # delete
          brute_ed(s1, s2, i-1, j-1)) # replace

# dynamic programming solution, where n, m - length of both strings
# time complexity: O(n*m)
# space complexity: O(n*m)
def levenshtein_distance(T1, T2):
    n, m = len(T1), len(T2)
    table = [[0]*(n+1) for i in range(m+1)]
    for i in range(m+1):
        for j in range(n+1):
            if i == 0:
                table[i][j] = j
            elif j == 0:
                table[i][j] = i
            elif T1[j-1] == T2[i-1]:
                table[i][j] = table[i-1][j-1]
            else:
                table[i][j] = 1 + min(table[i][j-1], table[i-1][j], table[i-1][j-1])
    return table[-1][-1] 


### C-13.27

Design an efficient algorithm for the matrix chain multiplication problem that outputs a fully parenthesized expression for how to multiply the matrices in the chain using the minimum number of operations.

In [19]:
# well, I guess the text book aproach also works, and it has less code
# but here is mine
# time complexity: O(n^3)
# space complexity: O(n^2)

# very smart trick with loops in the middle: first compute |i - j| = 1, then |i - j| = 2, |i - j| = 3 etc.
# basically L is the number of matrixes multiplied
# input example: ex = [3, 2, 4, 2, 5] means A1(2x3) * A2(3x4) * A3(4x2), e.g. these are 4 dimensions
def matrix_chain(m):
    n = len(m)
    table = [[0]*n for i in range(n)]
    for L in range(2, n):
        for i in range(1, n-L+1):      
            j = i+L-1
            maximum = 123456789
            for k in range(i, j):
                if maximum > table[i][k] + table[k+1][j] + m[i-1]*m[k]*m[j]:
                    maximum = table[i][k] + table[k+1][j] + m[i-1]*m[k]*m[j]
            table[i][j] = maximum
    return table

# doesn't print out parenthesized expression
